# ニューラルネットワークを用いたテキスト分類

このノートブックでは、IMDBのレビューデータセットを用いて、いくつかのテキスト分類モデルを構築します。

## 準備

### インポート

In [1]:
import os
import re
import string

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, TextVectorization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

## データセットの用意

まずは、事前学習済み単語埋め込みとして[GloVe](https://nlp.stanford.edu/projects/glove/)、分類のデータセットとして[IMDBレビューデータセット](http://ai.stanford.edu/~amaas/data/sentiment/)をダウンロードしましょう。
このデータセットは、50,000本の映画レビューのテキストを含んでいます。学習用とテスト用で各25,000件ずつに分けられています。学習データとテストデータは均衡しており、否定的なレビューと肯定的なレビューが同数含まれています。


In [2]:
# GloVeのダウンロードと展開
!wget  https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d DATAPATH

--2021-12-28 09:54:31--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-28 09:54:32--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 41s  

2021-12-28 09:57:13 (5.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: DATAPATH/glove.6B.50d.txt  
  inflating: DATAPATH/glove.6B.100d.txt  
  infl

IMDBデータセットについてですが、ダウンロードして展開し、読み込むためのコードを書いてもよいのですが、今回はデータセットを簡単に使い始められるパッケージである[TensorFlow Datasets](https://www.tensorflow.org/datasets)から利用します。TensorFlow Datasetsを使ったほうが、シンプルに書けるからです。学習データの80%を学習用に使い、残りの20%を検証用に使うことにしましょう。

In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'train[80%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6XM9MQ/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6XM9MQ/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


データの中身を見てみましょう。各例は、映画のレビューを表すテキストと、それに対応するラベルから構成されています。テキストは前処理されていません。ラベルは、0または1の整数値で、0は否定的なレビュー、1は肯定的なレビューを表しています。


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

次に、各種定数を定義します。

In [6]:
BASE_DIR = 'DATAPATH'
GLOVE_PATH = os.path.join(BASE_DIR, 'glove.6B.100d.txt')

# モデルの学習に関わるハイパーパラメータ
MAX_SEQUENCE_LENGTH = 250
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 

## 前処理

では次に、[TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)レイヤーを使って、データの標準化、トークン化、ベクトル化を行います。

標準化は、テキストを前処理することであり、句読点やHTML要素の削除などが該当します。トークン化では、文字列をトークンに分割します。ベクトル化では、トークンを数値に変換して、ニューラルネットワークに入力できるようにします。これらの作業はすべてこのレイヤーで行えます。

上のデータを見るとわかるのですが、レビューには<br />のようなHTMLタグが含まれています。これらのタグは、TextVectorizationレイヤーのデフォルトの標準化機能（デフォルトではテキストを小文字に変換し、句読点を除去する）では除去されません。このため、HTMLを除去するカスタム標準化関数を作成します。

In [7]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_html = tf.strings.regex_replace(
        stripped_html,
        '[%s]' % re.escape(string.punctuation),
        ''
    )
    return cleaned_html

次に、TextVectorizationレイヤーを作成します。このレイヤーを使って、データの標準化、トークン化、ベクトル化を行います。`output_mode`をintに設定し、各トークンにユニークな整数のインデックスを作成します。また、`output_sequence_length`を設定することで、ネットワークの入力となる配列をパディングしたり切り詰めたりすることもできます。

デフォルトでは、空白でテキストを分割しますが、`split`に呼び出し可能なオブジェクトを渡すことで、動作をカスタマイズできます。詳細については、以下のドキュメントを参照してください。

- [tf.keras.layers.TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)

In [8]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=MAX_NUM_WORDS,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

レイヤーを作成したら、ボキャブラリを作成するために`adapt`を呼び出します。これにより、文字列から整数へのインデックスを構築することになります。


In [9]:
# adapt用にテキストだけのデータセットを作成
train_text = train_data.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

`adapt`したら変換してみましょう。テキストからインデックスの系列に変換されていることがわかります。

In [10]:
print(train_examples_batch[0])
print(vectorize_layer(train_examples_batch))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(
[[  11   13   33 ...    0    0    0]
 [  10   25   74 ...    0    0    0]
 [4149 5732    2 ...    0    0    0]
 ...
 [   2   19    7 ...    0    0    0]
 [  10   62  112 ...    0    0    0]
 [ 247   11   28 ...    0    0    0]], shape=(10, 2

In [11]:
vectorize_layer.vocabulary_size()

20000

変換用のヘルパー関数を定義し、データセットに設定します。

In [12]:
def vectorize_text(text, label):
  return vectorize_layer(text), label

vectorize_text(train_examples_batch, train_labels_batch)

(<tf.Tensor: shape=(10, 250), dtype=int64, numpy=
 array([[  11,   13,   33, ...,    0,    0,    0],
        [  10,   25,   74, ...,    0,    0,    0],
        [4149, 5732,    2, ...,    0,    0,    0],
        ...,
        [   2,   19,    7, ...,    0,    0,    0],
        [  10,   62,  112, ...,    0,    0,    0],
        [ 247,   11,   28, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>)

In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_data.shuffle(
    buffer_size=len(train_data),
    reshuffle_each_iteration=True
  ).batch(32).map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_data.batch(32).map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_data.batch(32).map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
# 埋め込み行列の準備
# 最初に、単語のインデックスとベクトルのマッピングを作成
embeddings_index = {}
with open(os.path.join(GLOVE_PATH)) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))

# 埋め込み行列の準備
# 行は単語、列はGloVeから得た埋め込みに対応
num_words = min(MAX_NUM_WORDS, vectorize_layer.vocabulary_size()) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for i, word in enumerate(vectorize_layer.get_vocabulary()):
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    # 単語が見つからなければ、ゼロベクトルのまま
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Embedding層に事前学習済み単語埋め込みを読み込み
# 埋め込みを更新しないように、trainable=Falseを設定していることに注意
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False,
    mask_zero=True,
)

Found 400000 word vectors in Glove embeddings.


## モデルの学習

### 事前学習済み埋め込みを用いた1次元CNNモデル

In [15]:
# モデルの構築
cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(1, activation='sigmoid'))
cnnmodel.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

# モデルの学習
cnnmodel.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
)

# モデルの評価
score, acc = cnnmodel.evaluate(test_ds)
print('Test accuracy with CNN:', acc)

782/782 [==============================] - 26s 34ms/step - loss: 0.3813 - acc: 0.8298
Test accuracy with CNN: 0.8298400044441223


### 1次元CNNモデル

In [16]:
# モデルの構築
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(1, activation='sigmoid'))
cnnmodel.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# モデルの学習
cnnmodel.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

#　評価
score, acc = cnnmodel.evaluate(test_ds)
print('Test accuracy with CNN:', acc)

782/782 [==============================] - 28s 35ms/step - loss: 0.3135 - accuracy: 0.8635
Test accuracy with CNN: 0.8634799718856812


### LSTMモデル

In [17]:
# モデルの構築
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128, mask_zero=True))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(1, activation='sigmoid'))
rnnmodel.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# モデルの学習
rnnmodel.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

# モデルの評価
score, acc = rnnmodel.evaluate(test_ds)
print('Test accuracy with RNN:', acc)

782/782 [==============================] - 94s 120ms/step - loss: 0.6135 - accuracy: 0.6558
Test accuracy with RNN: 0.6558399796485901


### 事前学習済み埋め込みを用いたLSTMモデル

In [18]:
# モデルの構築
rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(1, activation='sigmoid'))
rnnmodel2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# モデルの学習
rnnmodel2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

# モデルの評価
score, acc = rnnmodel2.evaluate(test_ds)
print('Test accuracy with RNN:', acc)

782/782 [==============================] - 92s 117ms/step - loss: 0.5747 - accuracy: 0.7289
Test accuracy with RNN: 0.7289199829101562
